In [1]:
import pandas as pd
from string import digits
import re
import nltk

nltk.download('punkt')
nltk.download('popular')

from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.corpus import stopwords
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    | 

In [2]:
df = pd.read_csv("/content/drive/MyDrive/University/datasets2020/datasets/q1/train.csv")
#df = pd.read_csv("/mnt/5a5abb6b-4b3d-4ab8-bfea-9c7fc2ce23e6/linuxuser/Public/University/MSc/bigdata/datasets2020/datasets/q1/train.csv")
df.head()

,Id,Title,Content,Label
0,227464,"Netflix is coming to cable boxes, and Amazon i...",if you subscribe to one of three rinky-dink (...,Entertainment
1,244074,"Pharrell, Iranian President React to Tehran 'H...","pharrell, iranian president react to tehran '...",Entertainment
2,60707,Wildlife service seeks comments,the u.s. fish and wildlife service has reopen...,Technology
3,27883,Facebook teams up with Storyful to launch 'FB ...,the very nature of social media means it is o...,Technology
4,169596,Caesars plans US$880 mln New York casino,caesars plans us$880 mln new york casino jul ...,Business


In [3]:
df_test = pd.read_csv("/content/drive/MyDrive/University/datasets2020/datasets/q1/test_without_labels.csv")
#df_test = pd.read_csv("/mnt/5a5abb6b-4b3d-4ab8-bfea-9c7fc2ce23e6/linuxuser/Public/University/MSc/bigdata/datasets2020/datasets/q1/test_without_labels.csv")
df_test.head()

,Id,Title,Content
0,262120,Tracy Morgan upgraded to fair condition after ...,actor and comedian tracy morgan has been upgr...
1,175132,Smartphones Weigh on Samsung Electronics as Gu...,samsung electronics co ltd on tuesday issued u...
2,218739,FBI denies fumbling testimony on 'X-Men' direc...,michael f. egan iii said in a press conferenc...
3,253483,Bachelorette 2014 Spoilers: Week 3 Recap ??? E...,i am having mixed emotions for what is about ...
4,224109,Barack Obama honours Frankie Knuckles in lette...,u.s. president barack obama has paid a specia...


In [46]:
def blank_space(x):
  return re.sub('[^A-Za-z0-9]+', ' ', x)

def numbers(x):
  return re.sub(r'[0-9]+', '', x)

def standarize_sentence(x):
  return ''.join(''.join(word)[:2] for word in x) 

def apostrophe_words(x):
  Apos_dict={"'s":" is","n't":" not","'m":" am","'ll":" will", 
           "'d":" would","'ve":" have","'re":" are"} 
  for key,value in Apos_dict.items(): 
      if key in x: 
          return x.replace(key,value)
  return x

def split_words(x):
  return " ".join([word for word in re.split("([A-Z][a-z]+[^A-Z]*)",x) if word])

def shallow_cleaning(df):
  remove_digits = str.maketrans('', '', digits)
  df['Title Content'] = df['Title Content'].apply(lambda x: blank_space(x))
  df['Title Content'] = df['Title Content'].apply(lambda x: numbers(x))
  df['Title Content'] = df['Title Content'].apply(lambda x: split_words(x))
  df['Title Content'] = df['Title Content'].apply(lambda x: standarize_sentence(x))
  df['Title Content'] = df['Title Content'].apply(lambda x: apostrophe_words(x))
  df['Title Content'] = df['Title Content'].str.strip()
  df['Title Content'] = df['Title Content'].str.lower()
  return df

df['Title Content'] = pd.DataFrame(df['Title']+ ' ' + df['Content'])
df = shallow_cleaning(df)
display(df)

,Id,Title,Content,Label,Title Content
0,227464,"Netflix is coming to cable boxes, and Amazon i...",if you subscribe to one of three rinky-dink (...,Entertainment,netflix is coming to cable boxes and amazon i...
1,244074,"Pharrell, Iranian President React to Tehran 'H...","pharrell, iranian president react to tehran '...",Entertainment,pharrell iranian president react to tehran...
2,60707,Wildlife service seeks comments,the u.s. fish and wildlife service has reopen...,Technology,wildlife service seeks comments the u s fish a...
3,27883,Facebook teams up with Storyful to launch 'FB ...,the very nature of social media means it is o...,Technology,facebook teams up with storyful to launch fb...
4,169596,Caesars plans US$880 mln New York casino,caesars plans us$880 mln new york casino jul ...,Business,caesars plans us mln new york casino caesa...
...,...,...,...,...,...
111790,31462,Microsoft requires Office 2013 licensing for s...,in contrast to the muckle of special licenses...,Technology,microsoft requires office licensing for some...
111791,100821,Smallpox vials missing since 1950s found in la...,government workers at a research center near ...,Health,smallpox vials missing since s found in lab st...
111792,86181,Scientists May Have Just Discovered the Key to...,harvard scientists may have just unlocked the...,Health,scientists may have just discovered the k...
111793,256423,Justin Bieber to plead guilty to DUI,"justin bieber to plead guilty to duifri, 13 ju...",Entertainment,justin bieber to plead guilty to dui justin ...


In [47]:
df_test['Title Content'] = pd.DataFrame(df_test['Title']+ ' ' + df_test['Content'])
df_test = shallow_cleaning(df_test)
display(df_test)

,Id,Title,Content,Title Content,Predicted
0,262120,Tracy Morgan upgraded to fair condition after ...,actor and comedian tracy morgan has been upgr...,tracy morgan upgraded to fair condition after...,Business
1,175132,Smartphones Weigh on Samsung Electronics as Gu...,samsung electronics co ltd on tuesday issued u...,smartphones weigh on samsung electronics as...,Technology
2,218739,FBI denies fumbling testimony on 'X-Men' direc...,michael f. egan iii said in a press conferenc...,fbi denies fumbling testimony on x men direct...,Entertainment
3,253483,Bachelorette 2014 Spoilers: Week 3 Recap ??? E...,i am having mixed emotions for what is about ...,bachelorette spoilers week recap eric h...,Entertainment
4,224109,Barack Obama honours Frankie Knuckles in lette...,u.s. president barack obama has paid a specia...,barack obama honours frankie knuckles in le...,Entertainment
...,...,...,...,...,...
47907,50348,"BMW, Tesla meet to discuss standardizing elect...","june 16, 2014 by edward taylor reutersan emplo...",bmw tesla meet to discuss standardizing elect...,Technology
47908,255044,Harrison Ford has been filming the seventh Sta...,he may have helped save the galaxy from the ev...,harrison ford has been filming the seventh s...,Entertainment
47909,66502,"It's Games, Games, Games As Microsoft Plans To...",less than three months after microsoft had a ...,it s games games games as microsoft plan...,Technology
47910,10319,App Detail » Microsoft Excel for iPad,app description *** excel is ready for ipad p...,app detail microsoft excel for i pad app de...,Technology


In [48]:
porter = PorterStemmer()

df['Title Content'] = df['Title Content'].apply(lambda x: word_tokenize(x))
df['Title Content'] = df['Title Content'].apply(lambda x: [porter.stem(word) for word in x])
df['Title Content'] = df['Title Content'].apply(lambda x: ' '.join(str(word) for word in x))

df.head()


,Id,Title,Content,Label,Title Content
0,227464,"Netflix is coming to cable boxes, and Amazon i...",if you subscribe to one of three rinky-dink (...,Entertainment,netflix is come to cabl box and amazon is now ...
1,244074,"Pharrell, Iranian President React to Tehran 'H...","pharrell, iranian president react to tehran '...",Entertainment,pharrel iranian presid react to tehran happi a...
2,60707,Wildlife service seeks comments,the u.s. fish and wildlife service has reopen...,Technology,wildlif servic seek comment the u s fish and w...
3,27883,Facebook teams up with Storyful to launch 'FB ...,the very nature of social media means it is o...,Technology,facebook team up with story to launch fb newsw...
4,169596,Caesars plans US$880 mln New York casino,caesars plans us$880 mln new york casino jul ...,Business,caesar plan us mln new york casino caesar plan...


In [49]:
df_test['Title Content'] = df_test['Title Content'].apply(lambda x: word_tokenize(x))
df_test['Title Content'] = df_test['Title Content'].apply(lambda x: [porter.stem(word) for word in x])
df_test['Title Content'] = df_test['Title Content'].apply(lambda x: ' '.join(str(word) for word in x))

df_test.head()

,Id,Title,Content,Title Content,Predicted
0,262120,Tracy Morgan upgraded to fair condition after ...,actor and comedian tracy morgan has been upgr...,traci morgan upgrad to fair condit after crash...,Business
1,175132,Smartphones Weigh on Samsung Electronics as Gu...,samsung electronics co ltd on tuesday issued u...,smartphon weigh on samsung electron as guidanc...,Technology
2,218739,FBI denies fumbling testimony on 'X-Men' direc...,michael f. egan iii said in a press conferenc...,fbi deni fumbl testimoni on x men director bry...,Entertainment
3,253483,Bachelorette 2014 Spoilers: Week 3 Recap ??? E...,i am having mixed emotions for what is about ...,bachelorett spoiler week recap eric hill drama...,Entertainment
4,224109,Barack Obama honours Frankie Knuckles in lette...,u.s. president barack obama has paid a specia...,barack obama honour franki knuckl in letter to...,Entertainment


In [50]:
df.Label.unique()

array(['Entertainment', 'Technology', 'Business', 'Health'], dtype=object)

Creating Bag of Words

In [51]:
vectorizer=TfidfVectorizer(max_features=10000, stop_words=stopwords.words('english'), ngram_range=(1, 2))

temp = df['Title Content']
vectorized_content = vectorizer.fit_transform(temp)

train_bow = vectorized_content#.toarray()
#print(train_bow.shape)
train_bow

<111795x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 18266326 stored elements in Compressed Sparse Row format>

In [52]:
vectorizer=TfidfVectorizer(max_features=10000, stop_words=stopwords.words('english'), ngram_range=(1, 2))

temp = df_test['Title Content']
vectorized_test_content = vectorizer.fit_transform(temp)
#vectorized_test_content=vectorizer.transform(temp)

test_bow = vectorized_test_content#.toarray()
#print(test_bow.shape)
test_bow

<47912x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 7837987 stored elements in Compressed Sparse Row format>

In [53]:
def create_file(data, prediction, filename):
  res_df = pd.DataFrame(data)
  res_df['Predicted'] = prediction
  res_df.to_csv(filename+".csv", columns=['Id', 'Predicted'], index=False)
  return res_df

#/mnt/5a5abb6b-4b3d-4ab8-bfea-9c7fc2ce23e6/linuxuser/Public/University/MSc/bigdata/datasets2020/datasets/q1/"+

res_mean = pd.DataFrame([])
res_mean = res_mean.rename_axis('Statistic Measure', axis=1)
columnNum=0
def calculate_statistic_metrics(res, res_mean, columnName, columnNum):
  temp_df = pd.DataFrame([])
  accuracy_mean = pd.Series(res['test_accuracy'].mean(), name='Accuracy')
  precision_mean = pd.Series(res['test_precision_macro'].mean(), name='Precision')
  recall_mean = pd.Series(res['test_recall_macro'].mean(), name='Recall')
  F1_mean = pd.Series(res['test_f1_macro'].mean(), name='F-measure')
  temp_df = temp_df.append(accuracy_mean)
  temp_df = temp_df.append(precision_mean)
  temp_df = temp_df.append(recall_mean)
  temp_df = temp_df.append(F1_mean)
  res_mean[columnName] = temp_df[0]
  res_mean = res_mean.rename(columns={columnNum:columnName})
  return res_mean

In [54]:
def pickling_data(data_list,filename):
    output_file = open(filename,'wb')
    pickle.dump(data_list,output_file)   #function dump takes two args:the object you want to pickle and the file you want to save it in 
    output_file.close()

pickling_data(train_bow, 'train_bow')
pickling_data(test_bow, 'test_bow')

# **SVM**

In [13]:
from sklearn.svm import SVC, LinearSVC
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_validate

In [14]:
! wget http://195.134.67.98/documents/uci/test.csv

--2021-01-04 20:52:36--  http://195.134.67.98/documents/uci/test.csv
Connecting to 195.134.67.98:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 126607876 (121M) [text/csv]
Saving to: ‘test.csv’

test.csv            100%[===================>] 120.74M  6.36MB/s    in 21s     

2021-01-04 20:52:58 (5.88 MB/s) - ‘test.csv’ saved [126607876/126607876]



In [ ]:
X=train_bow
y=df['Label']

clf = LinearSVC(C=0.15, dual=False)

In [ ]:
scoring=['accuracy','precision_macro', 'recall_macro', 'f1_macro']
scores = cross_validate(clf, X, y, cv=5, n_jobs=4, scoring=scoring)
svm_res = pd.DataFrame.from_dict(scores)
res_mean = calculate_statistic_metrics(svm_res, res_mean, 'SVM (BoW)', columnNum)
display(res_mean)
res_mean.to_csv('evaluationResults.csv')

Statistic Measure,SVM (BoW)
Accuracy,0.966117
Precision,0.964982
Recall,0.961131
F-measure,0.963014


Make prediction for the test data with svm model

In [ ]:
y=df['Label']
clf = LinearSVC(C=0.15, dual=False)
clf.fit(X, y)
y_pred_test = clf.predict(test_bow)
svm_bow_pred_df = create_file(df_test, y_pred_test, 'svm_bow_pred')
display(svm_bow_pred_df)

,Id,Title,Content,Title Content,Predicted
0,262120,Tracy Morgan upgraded to fair condition after ...,actor and comedian tracy morgan has been upgr...,traci morgan upgrad to fair condit after crash...,Entertainment
1,175132,Smartphones Weigh on Samsung Electronics as Gu...,samsung electronics co ltd on tuesday issued u...,smartphon weigh on samsung electron as guidanc...,Entertainment
2,218739,FBI denies fumbling testimony on 'X-Men' direc...,michael f. egan iii said in a press conferenc...,fbi deni fumbl testimoni on x men director bry...,Entertainment
3,253483,Bachelorette 2014 Spoilers: Week 3 Recap ??? E...,i am having mixed emotions for what is about ...,bachelorett spoiler week recap eric hill drama...,Entertainment
4,224109,Barack Obama honours Frankie Knuckles in lette...,u.s. president barack obama has paid a specia...,barack obama honour franki knuckl in letter to...,Business
...,...,...,...,...,...
47907,50348,"BMW, Tesla meet to discuss standardizing elect...","june 16, 2014 by edward taylor reutersan emplo...",bmw tesla meet to discuss standard electr car ...,Entertainment
47908,255044,Harrison Ford has been filming the seventh Sta...,he may have helped save the galaxy from the ev...,harrison ford ha been film the seventh star wa...,Technology
47909,66502,"It's Games, Games, Games As Microsoft Plans To...",less than three months after microsoft had a ...,it s game game game as microsoft plan to close...,Entertainment
47910,10319,App Detail » Microsoft Excel for iPad,app description *** excel is ready for ipad p...,app detail microsoft excel for i pad app descr...,Business


In [ ]:
#! wget http://195.134.67.98/documents/uci/test.csv
df_test_labels=pd.read_csv("test.csv")
y = df_test_labels['Label']

print(classification_report(y,y_pred_test))

--2021-01-03 14:39:45--  http://195.134.67.98/documents/uci/test.csv
Connecting to 195.134.67.98:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 126607876 (121M) [text/csv]
Saving to: ‘test.csv.1’

test.csv.1          100%[===================>] 120.74M  3.27MB/s    in 39s     

2021-01-03 14:40:25 (3.06 MB/s) - ‘test.csv.1’ saved [126607876/126607876]

               precision    recall  f1-score   support

     Business       0.31      0.24      0.27     10910
Entertainment       0.45      0.77      0.57     19171
       Health       0.47      0.08      0.14      4943
   Technology       0.49      0.21      0.29     12888

     accuracy                           0.43     47912
    macro avg       0.43      0.33      0.32     47912
 weighted avg       0.43      0.43      0.38     47912



# **Random Forest**

In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import cross_validate
from sklearn.metrics import classification_report

In [16]:
X=train_bow
y=df['Label']

clf_Rf = RandomForestClassifier()

In [ ]:
scoring=['accuracy','precision_macro', 'recall_macro', 'f1_macro']
scores = cross_validate(clf_Rf, X, y, cv=5, n_jobs=4, scoring=scoring)
rf_res = pd.DataFrame.from_dict(scores)
res_mean = calculate_statistic_metrics(rf_res, res_mean, 'Random Forest (BoW)', columnNum)
display(res_mean)
res_mean.to_csv('evaluationResults')

Statistic Measure,SVM (BoW),Random Forest (BoW)
Accuracy,0.966117,0.938280
Precision,0.964982,0.938853
Recall,0.961131,0.925991
F-measure,0.963014,0.932104


Make label prediction for test data with bow

In [ ]:
clf_Rf = RandomForestClassifier()
clf_Rf.fit(X, y)
y_pred = clf_Rf.predict(test_bow)
rf_bow_pred_df = create_file(df_test, y_pred, 'rf_bow_pred')
display(rf_bow_pred_df)

,Id,Title,Content,Title Content,Predicted
0,262120,Tracy Morgan upgraded to fair condition after ...,actor and comedian tracy morgan has been upgr...,traci morgan upgrad to fair condit after crash...,Entertainment
1,175132,Smartphones Weigh on Samsung Electronics as Gu...,samsung electronics co ltd on tuesday issued u...,smartphon weigh on samsung electron as guidanc...,Technology
2,218739,FBI denies fumbling testimony on 'X-Men' direc...,michael f. egan iii said in a press conferenc...,fbi deni fumbl testimoni on x men director bry...,Entertainment
3,253483,Bachelorette 2014 Spoilers: Week 3 Recap ??? E...,i am having mixed emotions for what is about ...,bachelorett spoiler week recap eric hill drama...,Entertainment
4,224109,Barack Obama honours Frankie Knuckles in lette...,u.s. president barack obama has paid a specia...,barack obama honour franki knuckl in letter to...,Entertainment
...,...,...,...,...,...
47907,50348,"BMW, Tesla meet to discuss standardizing elect...","june 16, 2014 by edward taylor reutersan emplo...",bmw tesla meet to discuss standard electr car ...,Entertainment
47908,255044,Harrison Ford has been filming the seventh Sta...,he may have helped save the galaxy from the ev...,harrison ford ha been film the seventh star wa...,Business
47909,66502,"It's Games, Games, Games As Microsoft Plans To...",less than three months after microsoft had a ...,it s game game game as microsoft plan to close...,Technology
47910,10319,App Detail » Microsoft Excel for iPad,app description *** excel is ready for ipad p...,app detail microsoft excel for i pad app descr...,Entertainment


In [ ]:
#! wget http://195.134.67.98/documents/uci/test.csv
df_test_labels=pd.read_csv("test.csv")
y = df_test_labels['Label']

print(classification_report(y,y_pred_test))

               precision    recall  f1-score   support

     Business       0.31      0.24      0.27     10910
Entertainment       0.45      0.77      0.57     19171
       Health       0.47      0.08      0.14      4943
   Technology       0.49      0.21      0.29     12888

     accuracy                           0.43     47912
    macro avg       0.43      0.33      0.32     47912
 weighted avg       0.43      0.43      0.38     47912



# **SVD**

In [55]:
from sklearn.decomposition import TruncatedSVD

X=train_bow
y=df['Label']
Z=test_bow

svd = TruncatedSVD(n_components=100)
train_bow_svd = svd.fit_transform(X,y)
test_bow_svd = svd.fit_transform(Z)

For SVM

In [56]:
clf = LinearSVC(C=0.1)
scoring=['accuracy','precision_macro', 'recall_macro', 'f1_macro']
scores = cross_validate(clf, train_bow_svd, y, cv=5, n_jobs=4, scoring=scoring)
svm_res_svd = pd.DataFrame.from_dict(scores)
res_mean = calculate_statistic_metrics(svm_res_svd, res_mean, 'SVM (SVD)', columnNum)
display(res_mean)

Statistic Measure,SVM (SVD)
Accuracy,0.921052
Precision,0.918104
Recall,0.906139
F-measure,0.911822


In [57]:
#Predict for test set
y=df['Label']
clf = LinearSVC(C=0.1)
clf.fit(train_bow_svd,y)
y_pred_test = clf.predict(test_bow_svd)
SVM_pred_svd = create_file(df_test, y_pred_test, 'svm_bow_svd_pred')
SVM_pred_svd

,Id,Title,Content,Title Content,Predicted
0,262120,Tracy Morgan upgraded to fair condition after ...,actor and comedian tracy morgan has been upgr...,traci morgan upgrad to fair condit after crash...,Entertainment
1,175132,Smartphones Weigh on Samsung Electronics as Gu...,samsung electronics co ltd on tuesday issued u...,smartphon weigh on samsung electron as guidanc...,Business
2,218739,FBI denies fumbling testimony on 'X-Men' direc...,michael f. egan iii said in a press conferenc...,fbi deni fumbl testimoni on x men director bry...,Entertainment
3,253483,Bachelorette 2014 Spoilers: Week 3 Recap ??? E...,i am having mixed emotions for what is about ...,bachelorett spoiler week recap eric hill drama...,Entertainment
4,224109,Barack Obama honours Frankie Knuckles in lette...,u.s. president barack obama has paid a specia...,barack obama honour franki knuckl in letter to...,Entertainment
...,...,...,...,...,...
47907,50348,"BMW, Tesla meet to discuss standardizing elect...","june 16, 2014 by edward taylor reutersan emplo...",bmw tesla meet to discuss standard electr car ...,Technology
47908,255044,Harrison Ford has been filming the seventh Sta...,he may have helped save the galaxy from the ev...,harrison ford ha been film the seventh star wa...,Entertainment
47909,66502,"It's Games, Games, Games As Microsoft Plans To...",less than three months after microsoft had a ...,it s game game game as microsoft plan to close...,Entertainment
47910,10319,App Detail » Microsoft Excel for iPad,app description *** excel is ready for ipad p...,app detail microsoft excel for i pad app descr...,Technology


In [58]:
#! wget http://195.134.67.98/documents/uci/test.csv
df_test_labels=pd.read_csv("test.csv")
y_test = df_test_labels['Label']

print(classification_report(y_test,y_pred_test))

               precision    recall  f1-score   support

     Business       0.83      0.73      0.77     10910
Entertainment       0.86      0.94      0.90     19171
       Health       0.76      0.85      0.80      4943
   Technology       0.76      0.69      0.72     12888

     accuracy                           0.82     47912
    macro avg       0.80      0.80      0.80     47912
 weighted avg       0.82      0.82      0.81     47912



For Random Forest

In [59]:
clf_Rf = RandomForestClassifier(n_estimators=100, n_jobs=4)
y=df['Label']
scoring=['accuracy','precision_macro', 'recall_macro', 'f1_macro']
scores = cross_validate(clf_Rf, train_bow_svd, y, cv=5, n_jobs=4, scoring=scoring)
rf_res_svd = pd.DataFrame.from_dict(scores)
res_mean = calculate_statistic_metrics(rf_res_svd, res_mean, 'Random Forest (SVD)', columnNum)
display(res_mean)

Statistic Measure,SVM (SVD),Random Forest (SVD)
Accuracy,0.921052,0.949810
Precision,0.918104,0.948776
Recall,0.906139,0.940061
F-measure,0.911822,0.944255


In [60]:
#Predict for test set
clf_Rf = RandomForestClassifier(n_estimators=100, n_jobs=4)
clf_Rf.fit(train_bow_svd,y)
y_pred_test = clf_Rf.predict(test_bow_svd)
rf_pred_svd = create_file(df_test, y_pred_test, 'rf_bow_svd_pred')
display(rf_pred_svd)

,Id,Title,Content,Title Content,Predicted
0,262120,Tracy Morgan upgraded to fair condition after ...,actor and comedian tracy morgan has been upgr...,traci morgan upgrad to fair condit after crash...,Technology
1,175132,Smartphones Weigh on Samsung Electronics as Gu...,samsung electronics co ltd on tuesday issued u...,smartphon weigh on samsung electron as guidanc...,Technology
2,218739,FBI denies fumbling testimony on 'X-Men' direc...,michael f. egan iii said in a press conferenc...,fbi deni fumbl testimoni on x men director bry...,Entertainment
3,253483,Bachelorette 2014 Spoilers: Week 3 Recap ??? E...,i am having mixed emotions for what is about ...,bachelorett spoiler week recap eric hill drama...,Entertainment
4,224109,Barack Obama honours Frankie Knuckles in lette...,u.s. president barack obama has paid a specia...,barack obama honour franki knuckl in letter to...,Entertainment
...,...,...,...,...,...
47907,50348,"BMW, Tesla meet to discuss standardizing elect...","june 16, 2014 by edward taylor reutersan emplo...",bmw tesla meet to discuss standard electr car ...,Technology
47908,255044,Harrison Ford has been filming the seventh Sta...,he may have helped save the galaxy from the ev...,harrison ford ha been film the seventh star wa...,Entertainment
47909,66502,"It's Games, Games, Games As Microsoft Plans To...",less than three months after microsoft had a ...,it s game game game as microsoft plan to close...,Technology
47910,10319,App Detail » Microsoft Excel for iPad,app description *** excel is ready for ipad p...,app detail microsoft excel for i pad app descr...,Technology


In [61]:
#! wget http://195.134.67.98/documents/uci/test.csv
df_test_labels=pd.read_csv("test.csv")
y_test = df_test_labels['Label']

print(classification_report(y_test,y_pred_test))

               precision    recall  f1-score   support

     Business       0.87      0.78      0.82     10910
Entertainment       0.93      0.95      0.94     19171
       Health       0.77      0.86      0.81      4943
   Technology       0.81      0.81      0.81     12888

     accuracy                           0.86     47912
    macro avg       0.84      0.85      0.85     47912
 weighted avg       0.86      0.86      0.86     47912



# **My method**

In [62]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

In [63]:
X = train_bow
y = df['Label'] 

clf_nb = MultinomialNB()

In [64]:
scoring=['accuracy','precision_macro', 'recall_macro', 'f1_macro']
scores = cross_validate(clf_nb, X, y, cv=5, n_jobs=4, scoring=scoring)
nb_res = pd.DataFrame.from_dict(scores)
res_mean = calculate_statistic_metrics(nb_res, res_mean, 'My Method', columnNum)
#res_0.94104mean.to_csv('/mnt/5a5abb6b-4b3d-4ab8-bfea-9c7fc2ce23e6/linuxuser/Public/University/MSc/bigdata/datasets2020/datasets/q1/evaluationResults')
res_mean.to_csv('evaluationResults')
display(res_mean)

Statistic Measure,SVM (SVD),Random Forest (SVD),My Method
Accuracy,0.921052,0.949810,0.933092
Precision,0.918104,0.948776,0.926662
Recall,0.906139,0.940061,0.924815
F-measure,0.911822,0.944255,0.925701


In [65]:
clf_nb.fit(X,y)
y_pred_test = clf_nb.predict(test_bow)
rf_pred_svd = create_file(df_test, y_pred_test, 'nvb_pred')
display(rf_pred_svd)

,Id,Title,Content,Title Content,Predicted
0,262120,Tracy Morgan upgraded to fair condition after ...,actor and comedian tracy morgan has been upgr...,traci morgan upgrad to fair condit after crash...,Entertainment
1,175132,Smartphones Weigh on Samsung Electronics as Gu...,samsung electronics co ltd on tuesday issued u...,smartphon weigh on samsung electron as guidanc...,Business
2,218739,FBI denies fumbling testimony on 'X-Men' direc...,michael f. egan iii said in a press conferenc...,fbi deni fumbl testimoni on x men director bry...,Technology
3,253483,Bachelorette 2014 Spoilers: Week 3 Recap ??? E...,i am having mixed emotions for what is about ...,bachelorett spoiler week recap eric hill drama...,Entertainment
4,224109,Barack Obama honours Frankie Knuckles in lette...,u.s. president barack obama has paid a specia...,barack obama honour franki knuckl in letter to...,Entertainment
...,...,...,...,...,...
47907,50348,"BMW, Tesla meet to discuss standardizing elect...","june 16, 2014 by edward taylor reutersan emplo...",bmw tesla meet to discuss standard electr car ...,Business
47908,255044,Harrison Ford has been filming the seventh Sta...,he may have helped save the galaxy from the ev...,harrison ford ha been film the seventh star wa...,Technology
47909,66502,"It's Games, Games, Games As Microsoft Plans To...",less than three months after microsoft had a ...,it s game game game as microsoft plan to close...,Entertainment
47910,10319,App Detail » Microsoft Excel for iPad,app description *** excel is ready for ipad p...,app detail microsoft excel for i pad app descr...,Business


In [66]:
#! wget http://195.134.67.98/documents/uci/test.csv
df_test_labels=pd.read_csv("test.csv")
y = df_test_labels['Label']

print(classification_report(y,y_pred_test))

               precision    recall  f1-score   support

     Business       0.48      0.39      0.43     10910
Entertainment       0.57      0.74      0.64     19171
       Health       0.39      0.29      0.33      4943
   Technology       0.53      0.44      0.48     12888

     accuracy                           0.53     47912
    macro avg       0.49      0.46      0.47     47912
 weighted avg       0.52      0.53      0.52     47912

